This is a demo notebook for the [bcbcpy](https://github.com/aheritianad/BootCamp-BlockChain-and-Python/) package.

In [1]:
try:
    import bcbcpy
except ModuleNotFoundError:
    import os, sys
    sys.path.append(os.path.abspath("../src/"))
    import bcbcpy

# Cryptography


In [2]:
from bcbcpy import crypto

## Hashing


### `Hash` function


$$\mathfrak{h}: X \to H$$

- $\mathfrak{h}(x) = \mathfrak{h}(y)$ *only if* $x=y$.
- given $h\in H$, it is *not easy* to find $x\in X$ such that $\mathfrak{h}(x) = h$.

In some sense, $\mathfrak{h}(x)$ is a signature o $x$.

In [3]:
text_to_be_hashed = "Hello World"
first_hash = crypto.hash_function(text_to_be_hashed, hash_name=crypto.HashName.sha256)
first_hash


'1370eeaaba7a6c7a234b1f82cc3b6d013a0088fd5e16408300f05b28b0015463'

In [4]:
crypto.hash_function(text_to_be_hashed + "!")


'e59f8bdf1305e382a4919ccefd613d3eebae612aa4c443f3af2d65663de3b075'

In [5]:
crypto.hash_function("Hello World!")

'e59f8bdf1305e382a4919ccefd613d3eebae612aa4c443f3af2d65663de3b075'

In [6]:
long_text = """
Hello everyone!

Welcome to this tutorial.
I promise that I will do my best to help you.

It will be interactive, so please participate as much as you can.

Please do not hesitate to ask if there you have questions.

Hope you will enjoy it!

Cheers!

Heritiana.
"""


In [7]:
hash_long = crypto.hash_function(long_text)
hash_long


'0c8359860bef1b26f811bf8448c546c2241c7c695f9b8ee76c1a8bf2d5b2dff3'

In [8]:
len(hash_long) == len(first_hash)


True

### Validation


In [9]:
difficulty = 4
crypto.is_valid_hash(first_hash, difficulty)


False

In [10]:
fake_hash = "0000" + first_hash[4:]
fake_hash

'0000eeaaba7a6c7a234b1f82cc3b6d013a0088fd5e16408300f05b28b0015463'

In [11]:
crypto.is_valid_hash(fake_hash, difficulty)


True

### Nonce


In [12]:
second_hash, nonce = crypto.hash_nonce_initializer(
    text_to_be_hashed, difficulty=difficulty
)
second_hash

'0000bfe6af4232f78b0c8eba37a6ba6c17b9b8671473b0b82305880be077edd9'

In [13]:
nonce


107105

In [14]:
crypto.is_valid_hash(second_hash, difficulty)


True

In [15]:
third_hash = crypto.hash_function(text_to_be_hashed, nonce=nonce)
third_hash


'0000bfe6af4232f78b0c8eba37a6ba6c17b9b8671473b0b82305880be077edd9'

In [16]:
second_hash == third_hash


True

In [17]:
forth_hash = crypto.hash_function(text_to_be_hashed, nonce, nonce="")  # type:ignore
second_hash == forth_hash

True

### Q: What if such `nonce` does not exist?


**Think about it.**


## Crypto


### `Encryption` & `Decryption`


$$f\circ f^{-1} (x) = f^{-1}\circ f(x) = x,\ \forall x$$


In [18]:
encrypt = lambda x: 2 * x - 1
decrypt = lambda x: 0  # TODO to fill

In [19]:
import random
try:
    for i in range(20):
        x = random.randint(-5000, 5000)
        assert (
            encrypt(decrypt(x)) == decrypt(encrypt(x)) == x
        ), f"Oops! fail at {i+1}-th attempt for x = {x}."
except AssertionError as e:
    print(e)
else:
    print("Congrats! I passes the test.")

Oops! fail at 1-th attempt for x = 2125.


In [20]:
clear_message = (
    "This is a clear message to encrypt and decrypt. I add this to make it longer."
)

### Types


#### Key


$\mathcal{F}=\{ f_\theta \:\ \theta \in \Theta\}$

$\left(f_{\theta}\right) ^{-1} = f_{\theta^\prime}$ for some $\theta^\prime \in \Theta$

$\theta \in \Theta$ : key


#### Symmetric


Given $f_\theta$ or $\theta\in \Theta$ (thus $f_\theta$), one can construct  $\left(f_\theta\right)^{-1}$.

##### Cesar


In [21]:
cesar_key = crypto.CesarKey.generate_key(max_length=2)
print(cesar_key)


CesarKey(1, -1)


In [22]:
crypto.CesarKey.compute_inverse(6)

-6

In [23]:
encr_cesar = cesar_key.encrypt(clear_message)

print(
    f"""
original:
========
{clear_message}

Encrypted:
========
{encr_cesar}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Encrypted:
Uijt!jt!b!dmfbs!nfttbhf!up!fodszqu!boe!efdszqu/!J!bee!uijt!up!nblf!ju!mpohfs/



In [24]:
decr = cesar_key.decrypt(encr_cesar)

print(
    f"""
original:
========
{clear_message}

Decrypted:
=========
{decr}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



##### Permutation


In [25]:
perm_key = crypto.PermutationKey.generate_key(length=3, n_runs=1)
print(perm_key)


PermutationKey([3, 1, 2], [2, 3, 1])


In [26]:
encr_perm = perm_key.encrypt(clear_message)

print(
    f"""
original:
========
{clear_message}

Encrypted:
========
{encr_perm}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Encrypted:
iThis as l creae mass ge tocenpryat  ndcdepry t.aI  ddithts mo eakt io lengr.



In [27]:
decr = perm_key.decrypt(encr_perm)
print(
    f"""
original:
========
{clear_message}

Decrypted:
=========
{decr}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



##### Adding noise


In [28]:
from bcbcpy import utils


In [29]:
noisy_message = utils.add_noises(clear_message)
noisy_message


'TLhciist ViMsW %a/ uc8lAe[aVr| @mbe:s-s:aGg|ei ntLog 6ejn,c[rOy&p8tI \na{n3dy Fdee5c!ryy\tpqt<." PI$ 0a=d/d? ,t3hJiLsJ 9tBo, %mEa8kzer 0iOt. tlNovnFg\'e.r0.\n'

In [30]:
utils.remove_noises(noisy_message)


'This is a clear message to encrypt and decrypt. I add this to make it longer.'

In [31]:
encr_noisy = cesar_key.encrypt(noisy_message)
decr_noisy = cesar_key.decrypt(encr_noisy)
decr_without_noise = utils.remove_noises(decr_noisy)

print(
    f"""
\t\t--- CESAR --
Original:
========
{clear_message}

Noisy message:
=============
{noisy_message}

Noisy Encrypted:
===============
{encr_noisy}

Noisy Decrypted:
===============
{decr_noisy}


Cleaned Decrypted:
=================
{decr_without_noise}
"""
)


		--- CESAR --
Original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Noisy message:
TLhciist ViMsW %a/ uc8lAe[aVr| @mbe:s-s:aGg|ei ntLog 6ejn,c[rOy&p8tI 
a{n3dy Fdee5c!ryy	pqt<." PI$ 0a=d/d? ,t3hJiLsJ 9tBo, %mEa8kzer 0iOt. tlNovnFg'e.r0.


Noisy Encrypted:
UMidjjtu!WjNtX!&b0!vd9mBf\bWs}!Ancf;t.t;bHh}fj!ouMph!7fko-d\sPz'q9uJ! b|o4ez!Geff6d"szz
qru=/#!QJ%!1b>e0e@!-u4iKjMtK!:uCp-!&nFb9l{fs!1jPu/!umOpwoGh(f/s1/ 

Noisy Decrypted:
TLhciist ViMsW %a/ uc8lAe[aVr| @mbe:s-s:aGg|ei ntLog 6ejn,c[rOy&p8tI 
a{n3dy Fdee5c!ryy	pqt<." PI$ 0a=d/d? ,t3hJiLsJ 9tBo, %mEa8kzer 0iOt. tlNovnFg'e.r0.



Cleaned Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



In [32]:
encr_noisy = perm_key.encrypt(noisy_message)
decr_noisy = perm_key.decrypt(encr_noisy)
decr_without_noise = utils.remove_noises(decr_noisy)

print(
    f"""
\t\t--- PERMUTATION ---
Original:
========
{clear_message}

Noisy message:
=============
{noisy_message}

Noisy Encrypted:
===============
{encr_noisy}

Noisy Decrypted:
===============
{decr_noisy}


Cleaned Decrypted:
=================
{decr_without_noise}
"""
)


		--- PERMUTATION ---
Original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Noisy message:
TLhciist ViMsW %a/ uc8lAe[aVr| @mbe:s-s:aGg|ei ntLog 6ejn,c[rOy&p8tI 
a{n3dy Fdee5c!ryy	pqt<." PI$ 0a=d/d? ,t3hJiLsJ 9tBo, %mEa8kzer 0iOt. tlNovnFg'e.r0.


Noisy Encrypted:
hTLici stMVi sW/%ac uA8lae[|Vrm @:bess-G:aeg|ni otL6g nej[,cyrO8&p tI{
adn3Fy ede!5cyryq	p.t<P"  I$=0add/,? ht3LJi sJB9t o,E%mka8rzei 0.Otl tvNognF.'e.r0


Noisy Decrypted:
TLhciist ViMsW %a/ uc8lAe[aVr| @mbe:s-s:aGg|ei ntLog 6ejn,c[rOy&p8tI 
a{n3dy Fdee5c!ryy	pqt<." PI$ 0a=d/d? ,t3hJiLsJ 9tBo, %mEa8kzer 0iOt. tlNovnFg'e.r0.



Cleaned Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



#### Asymmetric


##### RSA


Security: number factorization complexity


In [33]:
rsa_key = crypto.rsa_key_demo()
print(rsa_key)


RSAPairKeys((7313, 73), HIDDEN_KEY)


In [34]:
encr_rsa = rsa_key.encrypt(clear_message)
encr_rsa


'[\n"#dYx[rp+uBpUUPf=KQ\\tsSl U,eut)55j7[c\'\\nq~Ao jgT;\'/WAi<z~pG]\'}1:Qf/>ddc18(re+ FN$"\n]'

In [35]:
rsa_key = crypto.rsa_key_demo(chunk_size=5)
encr_rsa=rsa_key.encrypt(clear_message)
encr_rsa

'[\n"Go,5<j",\n"2aa8\\"m",\n">ZBd$y",\n"?Tb-w\\n",\n"Gsk,L3",\n"\\n4g-\\\\W",\n"nzkvS",\n"Gt#HL ",\n"AOGz*S",\n"&E<;^i",\n"Gsy4&w",\n"IOix|<",\n"\\n4h53Q",\n"0}AqX0",\n">U~Ne+",\n"x "\n]'

In [36]:
utils.txt2obj(encr_rsa)

['Go,5<j',
 '2aa8"m',
 '>ZBd$y',
 '?Tb-w\n',
 'Gsk,L3',
 '\n4g-\\W',
 'nzkvS',
 'Gt#HL ',
 'AOGz*S',
 '&E<;^i',
 'Gsy4&w',
 'IOix|<',
 '\n4h53Q',
 '0}AqX0',
 '>U~Ne+',
 'x ']

In [37]:
from bcbcpy.crypto import read_from_rsa_convert

In [38]:
read_from_rsa_convert(encr_rsa)


'Go,5<j2aa8"m>ZBd$y?Tb-w\nGsk,L3\n4g-\\WnzkvSGt#HL AOGz*S&E<;^iGsy4&wIOix|<\n4h53Q0}AqX0>U~Ne+x '

In [39]:
decr_rsa = rsa_key.decrypt(encr_rsa)
read_from_rsa_convert(decr_rsa)


'This is a clear message to encrypt and decrypt. I add this to make it longer.'

##### Diffie-Hellman

Security: discrete logarithm complexity


##### ECC


### Communication & Security


#### Nodes generation

In [40]:
from bcbcpy.node import Node


In [41]:
key = crypto.rsa_key_demo()
node = Node(key)
node.id

'user_386194'

In [42]:
node

In [43]:
class RepNode(Node):
    def __repr__(self) -> str:
        return f"{self.id} : {self._Node__keys}"  # type:ignore

In [44]:
akey = crypto.CesarKey.generate_key()
alice = RepNode(akey, "Alice")
alice


Alice : CesarKey(78, -78)

In [45]:
bkey =  crypto.PermutationKey.generate_key(10)
bob = RepNode(username="Bob", keys=bkey)
bob

Bob : PermutationKey([1, 3, 6, 10, 5, 2, 7, 9, 8, 4], [1, 6, 2, 10, 5, 3, 7, 9, 8, 4])

In [46]:
rkey = crypto.rsa_key_demo()
random_guy = RepNode(rkey)
random_guy

user_117681 : RSAPairKeys((7313, 73), HIDDEN_KEY)

#### Communication

In [47]:
sender = alice
receiver = bob

plain_message = (
    "Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)"
)

In [48]:
cipher = sender.encrypt(plain_message, key=receiver.pub)
cipher

')96270 438t5nc:(soc1hut i 0ota$  d10sm 0eesP eeelnialic.i s TA Bhslbo eo! lH'

In [49]:
receiver.decrypt(cipher)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

#### Attack

##### Reading attack


In [50]:
hacker = random_guy

In [51]:
crypto.rsa.read_from_rsa_convert(
    hacker.decrypt(cipher)
)

"+@+jvhj:P;9zPN4j)QE(1L1u~83`~SfU/ef'8E]OH<ql|Z2XJ-YVb>a-O7Y2y}5G|SVbbqN~e|`M"

In [52]:
hacker.encrypt(cipher, bob.pub)

'H elolBob!A Ti his sa ielcPle.eness d me 0$0 1 to0uahsticco 51t n:234(97806)'

##### Q: what if `bob.pub == bob.priv`?

##### Sending attack


In [53]:
scam_message = (
    "Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)"
)

In [54]:
scam_cipher= hacker.encrypt(scam_message, key=receiver.pub)
scam_cipher

')25941 783t6nc:(soc0hut i 0ota$  d10sm 0eesP eeelnialic.i s TA Bhslbo eo! lH'

In [55]:
receiver.decrypt(scam_cipher)

'Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)'

### Authentication


#### Encryption

In [56]:
signed_message = sender.sign(plain_message) # encrypt with sender.priv
cipher_signed = sender.encrypt(signed_message, key=receiver.pub)
cipher_signed

"<LIEJC3GFK&H vM;%!vD{'&3|3C!&t733wDC%\n3Cxx%c3xxx\t |t\t|vA|3%3gT3U{%\tu!3x!43\t["

In [57]:
sender.sign(plain_message) == sender.decrypt(plain_message)

True

In [58]:
with_noise = utils.add_noises(plain_message)
signed_with_noise = sender.sign(with_noise) 
cipher_signed_with_noise = sender.encrypt(signed_with_noise, key=receiver.pub)
cipher_signed_with_noise

'KkL=C<\t]K9J?G>HIkFFFEZ3G;D3MMS&O!p\' 9~vIv=%:3t]e|F{=!,3&JY&83XCACCvlDx7t\n\nx3:?3<w[%Wx &)3hx?xdt%!*\t$cKxfA3Yqv3|u3QT\t+z%||_|U%3Y"{Mg?u?43G.!}U!\t5!3]k\tQx['

##### Shortcut

In [59]:
cipher_signed == sender.sends(plain_message, _to=receiver.pub)

True

In [60]:
cipher_signed == sender.sends(plain_message, _to=receiver)

True

In [61]:
cipher_signed_with_noise_bis = sender.sends(plain_message, _to=receiver, _with_noises=True)
cipher_signed_with_noise_bis

'NoIsYnOiSyz)LSC<5?K%J GmHIhGFOE"3\';D*iMU&-!U\' *Pv1vO%R3t\t;|5{[!$3&I;&@3,C?CC@gDD7A\n?x3If3kwJ%ix 7K3uxFx3t%-g\t\nc5x8A3gSv=|Y3?T\thv%%|)|N%3?N{\tg-u<43>+!cUs\t*!306\tWx[NoIsYnOiSy'

In [62]:
cipher_signed_with_noise_bis[10:-10] # trim NoIsYnOiSy

'z)LSC<5?K%J GmHIhGFOE"3\';D*iMU&-!U\' *Pv1vO%R3t\t;|5{[!$3&I;&@3,C?CC@gDD7A\n?x3If3kwJ%ix 7K3uxFx3t%-g\t\nc5x8A3gSv=|Y3?T\thv%%|)|N%3?N{\tg-u<43>+!cUs\t*!306\tWx['

In [63]:
cipher_signed_with_noise == cipher_signed_with_noise_bis[10:-10]

False

##### Q: Do you know why is it `False`?

#### Decryption

In [64]:
signed_message = receiver.decrypt(cipher_signed)
receiver.encrypt(signed_message, key=sender.pub)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [65]:
signed_message_with_noise = receiver.decrypt(cipher_signed_with_noise)
message_with_noise = receiver.encrypt(signed_message_with_noise, key=sender.pub) # unsign
utils.remove_noises(message_with_noise)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [66]:
trimmed_cipher_signed_with_noise = cipher_signed_with_noise_bis[10:-10]
signed_message_with_noise = receiver.decrypt(trimmed_cipher_signed_with_noise)
message_with_noise = receiver.encrypt(signed_message_with_noise, key=sender.pub) # unsign
utils.remove_noises(message_with_noise)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

##### Shortcut

In [67]:
receiver.gets(cipher_signed, _from=sender.pub)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [68]:
utils.remove_noises(receiver.gets(cipher_signed_with_noise, _from=sender))

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [69]:
receiver.gets(cipher_signed_with_noise_bis, _from=sender)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

#### Attacks

##### Reading Attack


In [70]:
crypto.read_from_rsa_convert(hacker.decrypt(cipher_signed))

'1D9p|Rv=Rnl$m"=^.^72N}Sw#My6^SzJ($:}ORosMozM75!6BX!B;6w20;D0SKVBx2UBD[#b\'($x'

In [71]:
hacker.encrypt(cipher_signed, key=sender.pub)

')96270 438t5nc:(soc1hut i 0ota$  d10sm 0eesP eeelnialic.i s TA Bhslbo eo! lH'

In [72]:
hacker.encrypt(cipher_signed, key=receiver.pub)

"[3x\t!\tU!u4T3g|3{|%3%t3|x\tvc\txAx x%%3w3\nx3C7C3D3&!C't{%&|vv!3HD&3 MEFG;LJKCI<"

In [73]:
hacker.sends(cipher_signed, _to=receiver.pub)


']"$x(\n\'#B[bUx2BDVS;0K0w26D;!6XB!75MBzMRsoO:}$o(z6SJy#Mw^SN^2}.=^"7\\$Rlm=Rv|npD\n19"['

In [74]:
hacker.gets(cipher_signed, _from=alice)


'Hmp\t1&]i?c*?[YrZIp*K|K$\n;j@dq:f#K@g7vr\'j<?\\`:\\g:$"o#/Eo/(#d\n~(1~@8C/e\nB/1HqOuvrepmJ'

In [75]:
hacker.gets(cipher_signed, _from=bob)


']"$x(\n\'#B[bUx2BDVS;0K0w26D;!6XB!75MBzMRsoO:}$o(z6SJy#Mw^SN^2}.=^"7\\$Rlm=Rv|npD\n19"['

##### Sending Attack


In [76]:
print(f"""
Original:
========
{plain_message}

Scam:
=====
{scam_message}
"""
)


Original:
Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)

Scam:
=====
Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)



In [77]:
non_signed_scam = hacker.encrypt(scam_message, key=receiver.pub)

receiver.gets(non_signed_scam, _from=sender)

'5RYY\\n/\\OonAUV`nV`n.YVPR|n=YRN`Rn`R[QnZRnr\t~~~na\\naUV`nNPP\\b[a\'nv~&%$#"! \n\tw'

In [78]:
non_signed_scam = hacker.encrypt(scam_message, key=sender.pub)

receiver.gets(non_signed_scam, _from=sender)

'dpltom[rsnuHc=kqMI=NCN[[;OkINBl[k>k[MG[_[M?*H???F?=FiC[;C[MCB[M}[|F<I.FI\\?["'

In [79]:
signed_scam = hacker.sends(scam_message, _to=receiver)

receiver.gets(signed_scam, _from=sender)

'Hmp/uCII.}v!?x%CK%IIrSIp(2q$WJQ)--M<%Lv&j0L%a%d1N$b3)Qn*?8s\\9u%zW.{rs=-gT//WbIawggbpmJ'

##### Possible Weakness

In [80]:
_, leaked_sender_priv = akey  # key of alice = expected sender

fake_signed_scam = hacker.encrypt(scam_message, leaked_sender_priv)
cipher_fake_signed_scam = hacker.encrypt(fake_signed_scam, receiver.pub)
cipher_fake_signed_scam

"<EHLGD3JKF&I vM;%!vC{'&3|3C!&t733wDC%\n3Cxx%c3xxx\t |t\t|vA|3%3gT3U{%\tu!3x!43\t["

In [81]:
receiver.gets(cipher_fake_signed_scam, _from=sender)

'Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)'

# Blockchain


In [82]:
from bcbcpy import blockchain

## Recap: Data Structure

### List - Array

### Linked List

## Block

In [83]:
from bcbcpy.blockchain.block import InitialBlock, Block

### Initial Block

In [84]:
init_block = InitialBlock(initial_data="initial data")

In [85]:
init_block.data

'initial data'

In [86]:
init_block.hash

'00002d64bd7129caa2aa8ec458081519ac26034158e60962f6fd1428e22b1036'

In [87]:
init_block

{
    "prev_hash": "",
    "hash": "00002d64bd7129caa2aa8ec458081519ac26034158e60962f6fd1428e22b1036",
    "data": "initial data",
    "nonce": 26206
}

### Block Data

In [88]:
second_block = Block(data="second data", prev_block= init_block)
second_block

{
    "prev_hash": "00002d64bd7129caa2aa8ec458081519ac26034158e60962f6fd1428e22b1036",
    "hash": "ac19b9c98cc9a1655bd66662c552b8667c2140a451ce635b24aa2fdf34555aea",
    "data": "second data",
    "nonce": 0
}

In [89]:
second_block.is_valid()

False

In [90]:
second_block.mine()
second_block

{
    "prev_hash": "00002d64bd7129caa2aa8ec458081519ac26034158e60962f6fd1428e22b1036",
    "hash": "0000f1a7cc9b3335858fdc615b274e07568931de65e82fb991ac257e168b89c1",
    "data": "second data",
    "nonce": 12005
}

In [91]:
second_block.is_valid()

True

In [92]:
third_block = Block("third data", prev_block= second_block)
third_block

{
    "prev_hash": "0000f1a7cc9b3335858fdc615b274e07568931de65e82fb991ac257e168b89c1",
    "hash": "f94643bbf3edffb31b549ce63a5adbd4542b7e3b6d45f32ae69da4f9bc87b6f7",
    "data": "third data",
    "nonce": 0
}

In [93]:
third_block.mine()
third_block

{
    "prev_hash": "0000f1a7cc9b3335858fdc615b274e07568931de65e82fb991ac257e168b89c1",
    "hash": "0000eecdfce497ff979154b54ddb7f8fe0114683c0102afb900cd93497951e80",
    "data": "third data",
    "nonce": 2917
}

In [94]:
forth_block = Block("forth data", third_block)
forth_block.mine()
forth_block

{
    "prev_hash": "0000eecdfce497ff979154b54ddb7f8fe0114683c0102afb900cd93497951e80",
    "hash": "00003e75b774fcf431090692c595e06fa5d05bc7730a63673e4e4304eb4faf6a",
    "data": "forth data",
    "nonce": 101056
}

In [95]:
fifth_block = Block("fifth block", forth_block)
fifth_block.mine()
fifth_block

{
    "prev_hash": "00003e75b774fcf431090692c595e06fa5d05bc7730a63673e4e4304eb4faf6a",
    "hash": "0000d8cb89717a6b1904bf4f44aab11529b3ee378d0256620f608c3dd73694b9",
    "data": "fifth block",
    "nonce": 34705
}

In [96]:
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, True, True, True)

### Making some change in past data

In [97]:
third_block.data = "not third data"
third_block

{
    "prev_hash": "0000f1a7cc9b3335858fdc615b274e07568931de65e82fb991ac257e168b89c1",
    "hash": "66ad1a8ee995ddbcaaf131585222c10a396a960f0c441578d5613417fa5d107a",
    "data": "not third data",
    "nonce": 2917
}

In [98]:
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, False, False, False)

In [99]:
third_block.data = "third data"
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, True, True, True)

In [100]:
third_block.data = "not third data"
third_block.mine()
third_block.is_valid()

True

In [101]:
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, True, False, False)

In [102]:
third_block.data = "third data"
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, False, False, False)

## Blockchain

A submodule is available at 
```python
from bcbcpy.blockchain import chain
```

### Root

In [103]:
blockchain.RootChain()

{
    "root_block": {
        "hash": "0000a456e7b5a5eb059e721fb431436883143101275c4077f83fe70298f5623d",
        "data": ""
    }
}

In [104]:
initial_data = {"INITIAL AGREEMENT": {"Bob": 1000, "Alice": 500}}

initial_data_string = utils.obj2txt(initial_data, indent=4)
root_chain = blockchain.RootChain(initial_data_string, difficulty=4)

root_chain

{
    "root_block": {
        "hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
        "data": "{\n    \"INITIAL ..."
    }
}

### Chain

In [105]:
prev_block = root_chain.last_block
new_block = Block("new block", prev_block)
new_block

{
    "prev_hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
    "hash": "73095d20a662a26d26eec210c0dc51c34816098cdab05f941487ee5e8133d6ad",
    "data": "new block",
    "nonce": 0
}

---

```python
root_chain.add_block(
  new_block
)
```

---
```sh
 ---------------------------------------------------------------------------
 AssertionError                            Traceback (most recent call last)
 /var/folders/x4/45r2t3bx5l3dc_b3tm_yz7380000gn/T/ipykernel_26354/943714534.py in <cell line: 1>()
 ----> 1 root_chain.add_block(
       2     new_block
       3 )

 ~/Desktop/BootCamp-BlockChain-and-Python/src/bcbcpy/blockchain/chain.py in add_block(self, block)
      26             self.last_block is block.prev_block
      27         ), "Incompatible block to the head block of the chain."
 ---> 28         assert block.is_valid(), "Block is not valid."
      29         self.last_block = block
      30 

 AssertionError: Block is not valid.
 
```
---

In [106]:
new_block.mine() # validation
new_block

{
    "prev_hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
    "hash": "00007b581d241fea3fb8bf4c7c5c1941af9abb3a5ce05abb97b4521d34bab727",
    "data": "new block",
    "nonce": 2823
}

In [107]:
root_chain.add_block(
    new_block
)

In [108]:
root_chain

{
    "root_block": {
        "hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
        "data": "{\n    \"INITIAL ..."
    },
    "last_block": {
        "hash": "00007b581d241fea3fb8bf4c7c5c1941af9abb3a5ce05abb97b4521d34bab727",
        "data": "new block"
    }
}

In [109]:
chain = blockchain.Chain(root_chain=root_chain, info="local network")
chain

{
    "root_block": {
        "hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
        "data": "local network"
    }
}

## Transaction


In [110]:
from bcbcpy.blockchain.block import transaction

### Nodes setup

In [111]:
from bcbcpy.blockchain.node import Node


alice = Node(crypto.PermutationKey.generate_key(10), "Alice")
bob = Node(crypto.PermutationKey.generate_key(10), "Bob")
other_person_in_the_network = Node(crypto.PermutationKey.generate_key(10))

### Transaction Data

In [112]:
assets = 1000
sender = bob
receiver = alice

transaction_message = ("""
Hello Alice, 

This is 1000 that you asked me to send last time.

Bye,
Bob.
"""
)

In [113]:
receiver_id = receiver.id
receiver_pub = receiver.pub

prev_block = chain.last_block

In [114]:
transaction_data = transaction.TransactionData(
    assets, sender, receiver_id, receiver_pub, prev_block, transaction_message
)

print(transaction_data)

{
    "timestamp": "2023-05-02T08:52:24.306510",
    "sender_id": "Bob",
    "receiver_id": "Alice",
    "assets": 1000,
    "prev_hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
    "sender_confirmation": ",oe\\n}\n.Bbetd\\n\"ninsalt s\\\\ ey e  kB. etuo0 hm sao00h  nmtialn\\cAdyTs iiollteH s \\,\"e1n:\"n_ssme\\enoarasangit\"c  \n],t             9 1   \n   , ,2  \n   \n \n, 5       \n4,     7  ,\n     ,  \n   6  \n      ,8        \n,       0        1\n  3u :e\", la[uekb_\nvp_yrieec_r\" v0\n,00e 1  atess :\" sb\n,9\"\" 97 48227 dc61978a6d8fc4422ab90c92b3cc98aabac55dd50bec045010d002h\"has0\"_v: \"    e\n{pr"
}


### Transaction Block

In [115]:
transaction_block = blockchain.TransactionBlock(transaction_data)
transaction_block

{
    "prev_hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
    "hash": "a50260b9638bf511280628f38b3b51f319ff94bfbd537758714d0210782b4727",
    "data": "{\n    \"timestamp\": \"2023-05-02T08:52:24.306510\",\n    \"sender_id\": \"Bob\",\n    \"receiver_id\": \"Alice\",\n    \"assets\": 1000,\n    \"prev_hash\": \"000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9\",\n    \"sender_confirmation\": \",oe\\\\n}\\n.Bbetd\\\\n\\\"ninsalt s\\\\\\\\ ey e  kB. etuo0 hm sao00h  nmtialn\\\\cAdyTs iiollteH s \\\\,\\\"e1n:\\\"n_ssme\\\\enoarasangit\\\"c  \\n],t             9 1   \\n   , ,2  \\n   \\n \\n, 5       \\n4,     7  ,\\n     ,  \\n   6  \\n      ,8        \\n,       0        1\\n  3u :e\\\", la[uekb_\\nvp_yrieec_r\\\" v0\\n,00e 1  atess :\\\" sb\\n,9\\\"\\\" 97 48227 dc61978a6d8fc4422ab90c92b3cc98aabac55dd50bec045010d002h\\\"has0\\\"_v: \\\"    e\\n{pr\"\n}",
    "nonce": 0
}

### Verification


#### Anyone in the network

In [116]:
data = utils.txt2obj(transaction_block.data)
data

{'timestamp': '2023-05-02T08:52:24.306510',
 'sender_id': 'Bob',
 'receiver_id': 'Alice',
 'assets': 1000,
 'prev_hash': '000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9',
 'sender_confirmation': ',oe\\n}\n.Bbetd\\n"ninsalt s\\\\ ey e  kB. etuo0 hm sao00h  nmtialn\\cAdyTs iiollteH s \\,"e1n:"n_ssme\\enoarasangit"c  \n],t             9 1   \n   , ,2  \n   \n \n, 5       \n4,     7  ,\n     ,  \n   6  \n      ,8        \n,       0        1\n  3u :e", la[uekb_\nvp_yrieec_r" v0\n,00e 1  atess :" sb\n,9"" 97 48227 dc61978a6d8fc4422ab90c92b3cc98aabac55dd50bec045010d002h"has0"_v: "    e\n{pr'}

In [117]:
sender_confirmation=data["sender_confirmation"]
sender_confirmation

',oe\\n}\n.Bbetd\\n"ninsalt s\\\\ ey e  kB. etuo0 hm sao00h  nmtialn\\cAdyTs iiollteH s \\,"e1n:"n_ssme\\enoarasangit"c  \n],t             9 1   \n   , ,2  \n   \n \n, 5       \n4,     7  ,\n     ,  \n   6  \n      ,8        \n,       0        1\n  3u :e", la[uekb_\nvp_yrieec_r" v0\n,00e 1  atess :" sb\n,9"" 97 48227 dc61978a6d8fc4422ab90c92b3cc98aabac55dd50bec045010d002h"has0"_v: "    e\n{pr'

In [118]:
confirmation_string = other_person_in_the_network.encrypt(sender_confirmation, key= bob.pub)
confirmation = utils.txt2obj(confirmation_string)
confirmation

{'prev_hash': '000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9',
 'assets': 1000,
 'receiver_pub_key_value': [3, 10, 8, 6, 7, 4, 5, 2, 1, 9],
 'transaction_message': 'e,\n\n Hillois1esTlcA\n tyit0  h0admasu h0oon e   k etm.e a stlyB\nie\ndts\nB.,\neob'}

In [119]:
print(alice.pub)
print(chain.hash)

[3, 10, 8, 6, 7, 4, 5, 2, 1, 9]
000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9


#### Extra for Receiver

In [120]:
transaction_message = receiver.gets(confirmation["transaction_message"], _from=sender)
print(transaction_message)


Hello Alice, 

This is 1000 that you asked me to send last time.

Bye,
Bob.



### Block Validation

In [121]:
transaction_block.is_valid()

False

In [122]:
transaction_block.mine()
transaction_block

{
    "prev_hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
    "hash": "000045d22eb3915e2a225c4a94459a39f10883acc9f0a1824d526ede4e759d0d",
    "data": "{\n    \"timestamp\": \"2023-05-02T08:52:24.306510\",\n    \"sender_id\": \"Bob\",\n    \"receiver_id\": \"Alice\",\n    \"assets\": 1000,\n    \"prev_hash\": \"000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9\",\n    \"sender_confirmation\": \",oe\\\\n}\\n.Bbetd\\\\n\\\"ninsalt s\\\\\\\\ ey e  kB. etuo0 hm sao00h  nmtialn\\\\cAdyTs iiollteH s \\\\,\\\"e1n:\\\"n_ssme\\\\enoarasangit\\\"c  \\n],t             9 1   \\n   , ,2  \\n   \\n \\n, 5       \\n4,     7  ,\\n     ,  \\n   6  \\n      ,8        \\n,       0        1\\n  3u :e\\\", la[uekb_\\nvp_yrieec_r\\\" v0\\n,00e 1  atess :\\\" sb\\n,9\\\"\\\" 97 48227 dc61978a6d8fc4422ab90c92b3cc98aabac55dd50bec045010d002h\\\"has0\\\"_v: \\\"    e\\n{pr\"\n}",
    "nonce": 11004
}

In [123]:
transaction_block.is_valid()

True

### Adding to Chain

In [124]:
chain.add_block(transaction_block)
chain

{
    "root_block": {
        "hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
        "data": "local network"
    },
    "last_block": {
        "hash": "000045d22eb3915e2a225c4a94459a39f10883acc9f0a1824d526ede4e759d0d",
        "data": "{\n    \"timestam..."
    }
}

## Second Transaction


In [125]:
assets = 500
sender = alice
receiver = bob
prev_block = chain.last_block

transaction_message = """
Hi Bob, 

Please, I am sending you back the 500 because I already have free 500 in my account.

Best,
Alice.
"""

In [126]:
transaction_block = sender.make_transaction_block(
    assets=assets,
    to=receiver,
    prev_block=prev_block,
    transaction_message=transaction_message,
)

### Verification ...

In [127]:
data = utils.txt2obj(transaction_block.data)
data

{'timestamp': '2023-05-02T08:52:24.988258',
 'sender_id': 'Alice',
 'receiver_id': 'Bob',
 'assets': 500,
 'prev_hash': '000045d22eb3915e2a225c4a94459a39f10883acc9f0a1824d526ede4e759d0d',
 'sender_confirmation': 'ne}lcn."\\\nns\\nu\\Ai\\,oiBcy .tce  a0e mn0ter5vy ena lchae df retIb50se a  0k b acugaanenuhie PdIsesy ono,H nam\\ i\\\\"n bl eae:ms_",sBcnnasroeag ,t]\n "i t5         ,       1\n\n   8   ,\n 7  ,   \n  , 9\n      \n , 2       \n ,      0   1     \n   ,         \n3  6      , 4[" e:u \n ,y_lbkua e\nvcpeer_vi_ 0"0,5 r\nets as": e \nd 0"d ",s4e96d25 e 805fa9d71e81cf09c4824a349caa4321595329e5d0b040e25a"a0hs_"2h2" v  \ne  :{pr'}

In [128]:
sender_confirmation=data["sender_confirmation"]
confirmation_string = other_person_in_the_network.encrypt(sender_confirmation, key= sender.pub)
confirmation = utils.txt2obj(confirmation_string)
confirmation

{'prev_hash': '000045d22eb3915e2a225c4a94459a39f10883acc9f0a1824d526ede4e759d0d',
 'assets': 500,
 'receiver_pub_key_value': [6, 4, 3, 10, 2, 9, 7, 8, 1, 5],
 'transaction_message': '\n iB,b\nHo \nela,eIPs   msdnnaeigoyuabk  c eh 00bt5 euasI ace laedh vryaerfe5 0 e0  nma ciycotn.B\nsu\netA\nlec\n,i.'}

In [129]:
print(receiver.pub)
print(chain.hash)

[6, 4, 3, 10, 2, 9, 7, 8, 1, 5]
000045d22eb3915e2a225c4a94459a39f10883acc9f0a1824d526ede4e759d0d


In [130]:
transaction_message = receiver.gets(confirmation["transaction_message"], _from=sender)
print(transaction_message)


Hi Bob, 

Please, I am sending you back the 500 because I already have free 500 in my account.

Best,
Alice.



### Adding to Chain

In [131]:
transaction_block.mine() # validation
chain.add_block(transaction_block)

In [132]:
chain

{
    "root_block": {
        "hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
        "data": "local network"
    },
    "...": "...",
    "last_block": {
        "hash": "00000824894578feef54cd849e66091ea15cb710777a8e92655d14b03a4d0e41",
        "data": "{\n    \"timestam..."
    }
}